<a href="https://colab.research.google.com/github/Bhanupartap00/FocusFlow_App/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


In [ ]:
df1 = pd.read_csv("RedditNews.csv")
df2 = pd.read_csv("upload_DJIA_table.csv")

df = pd.merge(df1, df2, on="Date", how="inner")

df['label'] = df["Close"] - df["Open"]
df['label'] = df['label'].apply(lambda x: 1 if x > 0 else 0)

df = df[['News', 'label']]
print(df.sample(5))


                                                    News  label
41694                 b'How America is snooping on YOU.'      1
41158  b'Egypt is building an underground barrier wit...      0
9139   Israel ignoring deaths of Thai workers on farm...      1
35827  A new online game has taken the internet by st...      1
35487                     The Arab myth of western women      1


In [ ]:
train, val, test = np.split(
    df.sample(frac=1, random_state=42),
    [int(0.8 * len(df)), int(0.9 * len(df))]
)


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('label')
    text = df['News']

    ds = tf.data.Dataset.from_tensor_slices((text, labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False)
test_ds = df_to_dataset(test, shuffle=False)


In [ ]:
hub_layer = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder/4",
    trainable=False,
    dtype=tf.string
)


In [ ]:
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
embeddings = tf.keras.layers.Lambda(lambda x: hub_layer(x), output_shape=(512,))(input_text)
dense1 = tf.keras.layers.Dense(16, activation='relu')(embeddings)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense1)

model = tf.keras.Model(inputs=[input_text], outputs=output)

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
    jit_compile=False
)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5393 - loss: 0.6906 - val_accuracy: 0.5408 - val_loss: 0.6905
Epoch 2/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5454 - loss: 0.6876 - val_accuracy: 0.5286 - val_loss: 0.6914
Epoch 3/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5550 - loss: 0.6849 - val_accuracy: 0.5159 - val_loss: 0.6942
Epoch 4/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5650 - loss: 0.6828 - val_accuracy: 0.5137 - val_loss: 0.6959
Epoch 5/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5744 - loss: 0.6774 - val_accuracy: 0.5119 - val_loss: 0.6985
Epoch 6/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5831 - loss: 0.6741 - val_accuracy: 0.5151 - val_loss: 0.7000
Epoch 7/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5896 - loss: 0.6698 - val_accuracy: 0.5221 - val_loss: 0.7027
Epoch 8/15
1243/1243 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - accuracy: 0.5981 -

In [ ]:
model.evaluate(test_ds)
